# Network Visualization

This notebook demonstrates how to view **MASSpy** models on network maps using the [Escher](https://escher.github.io/) visualization tool <cite data-cite="KDragerE+15">(King et al., 2015)</cite>.

The **Escher** package must already be installed into the environment. To install **Escher**:

```python
pip install escher
```

## Viewing Models with Escher

The **MASSpy** package also comes with some example maps.

In [1]:
from os.path import join

import numpy as np

import mass
import mass.example_data

# Load the glycolysis and hemoglobin models, then merge them
glycolysis = mass.example_data.create_example_model("Glycolysis")
hemoglobin = mass.example_data.create_example_model("Hemoglobin")
model = glycolysis.merge(hemoglobin, inplace=False)

# Set the path to the map file
map_filepath = join(mass.example_data.MAPS_DIR, "RBC.glycolysis.map.json")

# To view the list of available maps, remove the semicolon
mass.example_data.view_example_maps();

['RBC.glycolysis.map.json',
 'multicompartment_map.json',
 'phosphate_trafficking_map.json',
 'sb2_RBC_map.json',
 'sb2_amp_salvage_network_map.json',
 'sb2_glycolysis_map.json',
 'sb2_pentose_phosphate_pathway_map.json',
 'simple_toy_map.json']

The primary object for viewing **Escher** maps is the `escher.Builder`, a Jupyter widget that can be viewed in a Jupyter notebook. 

In [2]:
import escher
from escher import Builder

# Turns off the warning message when leaving or refreshing this page.
# The default setting is False to help avoid losing work.
escher.rc['never_ask_before_quit'] = True

To load an existing map, the path to the JSON file of the **Escher** map is provided to the `map_json` argument of the  `Builder`. The `MassModel` can be loaded using the `model` argument.

In [3]:
escher_builder = Builder(
    model=model,
    map_json=map_filepath)

escher_builder

Builder(never_ask_before_quit=True)

## Mapping Data onto Escher
### Viewing Reaction Data

Reaction data can be displayed on the **Escher** map using a dictionary that contains reaction identifiers, and values to map onto reaction arrows. The `dict` can be provided to the `reaction_data` argument upon initialization of the builder.

For example, to display the steady state fluxes on the map:

In [4]:
initial_flux_data = {
    reaction.id: flux
    for reaction, flux in model.steady_state_fluxes.items()}

# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=model,
    map_json=map_filepath,
    reaction_data=initial_flux_data)

# Display map in notebook
escher_builder

Builder(never_ask_before_quit=True, reaction_data={'HEX1': 1.12, 'PGI': 1.1200000000000092, 'PFK': 1.119999999…

The color and size of the data scale can be altered by providing a tuple of at least two dictionaries. Each dictionary is considered a "stop" that defines the color and size at or near that particular value in the data set. The `type` key defines the type for the stop, the `color` key defines the color of the arrow, and the `size` key defines the thickness of the arrow.

In [5]:
# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=model,
    map_json=map_filepath,
    reaction_data=initial_flux_data,
    reaction_scale=(
        {"type": 'min', "color": 'green', "size": 5 },
        {"type": 'value', "value": 1.12, "color": 'purple', "size": 10},
        {"type": 'max', "color": 'blue', "size": 15 }),
)

# Display map in notebook
escher_builder

Builder(never_ask_before_quit=True, reaction_data={'HEX1': 1.12, 'PGI': 1.1200000000000092, 'PFK': 1.119999999…

### Viewing Metabolite Data

Metabolite data also can be displayed on an **Escher** map by using a dictionary containing metabolite identifiers, and values to map onto metabolite nodes. In addition to setting the attributes to apply upon initializing the builder, the attributes also can be set for a map after initialization.

For example, to display metabolite concentrations on the map:

In [6]:
initial_conc_data = {
    metabolite.id: round(conc, 8)
    for metabolite, conc in model.initial_conditions.items()}

# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=model,
    map_json=map_filepath, 
    metabolite_data=initial_conc_data)

# Display map in notebook
escher_builder

Builder(metabolite_data={'glc__D_c': 1.00000036, 'g6p_c': 0.04859998, 'f6p_c': 0.01979999, 'fdp_c': 0.01460002…

The secondary metabolites can be removed by setting `hide_secondary_metabolites` as `True` to provide a cleaner visualization of the primary metabolites in the network.

In [7]:
escher_builder.hide_secondary_metabolites = True

Note that changes made affect the already displayed map. Here, a preset scale is applied to the metabolite concentrations.

In [8]:
escher_builder.metabolite_scale_preset = "RdYlBu"

### Visualizing SBML models with Escher in Python

Suppose that we would like to visualize our SBML model on a network map as follows:
1. We would like to create this map with the **Escher** web-based API. 
2. We would like to view the model on the network map within in a Jupyter notebook using the **Escher** Python-based API.
3. We would like to display the value of forward rate constants for each reaction on the network map.

The JSON format is the preferred format for **Escher** to load models onto network maps ([read more here](https://escher.readthedocs.io/en/latest/escher_and_cobrapy.html#what-is-json-and-why-do-we-use-it)). Therefore, we must convert models between SBML and JSON formats to achieve our goal.

**Note:** The models and maps used in the following example are also available in the example data.

In [9]:
import mass.io

Fortunately, the [mass.io](../autoapi/mass/io/index.rst) submodule is capable of exporting such models.

First the SBML model is loaded using the ``mass.io.sbml`` submodule. The model is then exported to a JSON format using the ``mass.io.json`` submodule for use in the [Escher web-based API](https://escher.github.io/).

In [10]:
# Define path to SBML model
path_to_sbml_model = join(mass.example_data.MODELS_DIR, "Simple_Toy.xml")

# Load SBML model
model = mass.io.sbml.read_sbml_model(path_to_sbml_model)

# Export as JSON
path_to_json_model = "./Simple_Toy.json"
mass.io.json.save_json_model(model, filename=path_to_json_model)

Suppose that we have now created our map using the **Escher** web-based API and saved it as the file "simple_toy_map.json". To display the map with the model:

In [11]:
# Define path to Escher map
path_to_map = join(mass.example_data.MAPS_DIR, "simple_toy_map.json")
escher_builder = Builder(
    model_json=path_to_json_model,
    map_json=path_to_map)
escher_builder

Builder(never_ask_before_quit=True)

Finally the forward rate constant data from the ``MassModel`` object is added to the map:

In [12]:
escher_builder.reaction_data = dict(zip(
    model.reactions.list_attr("id"),
    model.reactions.list_attr("forward_rate_constant")
))

## Additional Examples
For additional information and examples on how to visualize networks and **MASSpy** models using **Escher**, see the following: 

* [Animating Simulations with Escher](../gallery/visualization/animating_simulations.ipynb)